# Natural Language Processing 

Importing the dataset

In [1]:
dataset_original = read.delim('Restaurant_Reviews.tsv', quote = '', stringsAsFactors = FALSE)
head(dataset_original)

Review,Liked
Wow... Loved this place.,1
Crust is not good.,0
Not tasty and the texture was just nasty.,0
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.,1
The selection on the menu was great and so were the prices.,1
Now I am getting angry and I want my damn pho.,0


Cleaning the texts

In [2]:
library(tm)
library(SnowballC)
corpus = VCorpus(VectorSource(dataset_original$Review))
corpus = tm_map(corpus, content_transformer(tolower))
corpus = tm_map(corpus, removeNumbers)
corpus = tm_map(corpus, removePunctuation)
corpus = tm_map(corpus, removeWords, stopwords())
corpus = tm_map(corpus, stemDocument)
corpus = tm_map(corpus, stripWhitespace)

Warning message:
"package 'tm' was built under R version 3.4.3"Loading required package: NLP


Creating the Bag of Words model

In [3]:
dtm = DocumentTermMatrix(corpus)
dtm = removeSparseTerms(dtm, 0.999)
dataset = as.data.frame(as.matrix(dtm))
dataset$Liked = dataset_original$Liked

Encoding the target feature as factor

In [4]:
dataset = read.csv('Social_Network_Ads.csv')
dataset = dataset[3:5]

Warning message in file(file, "rt"):
"cannot open file 'Social_Network_Ads.csv': No such file or directory"

ERROR: Error in file(file, "rt"): cannot open the connection


In [5]:
dataset$Liked = factor(dataset$Liked, levels = c(0, 1))

In [6]:
head(dataset)

absolut,acknowledg,actual,ago,almost,also,although,alway,amaz,ambianc,...,wow,wrap,wrong,year,yet,youd,your,yummi,zero,Liked
0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Splitting the dataset into the Training set and Test set

In [7]:
library(caTools)
set.seed(123)
split = sample.split(dataset$Liked, SplitRatio = 0.8)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

Warning message:
"package 'caTools' was built under R version 3.4.3"

Fitting Random Forest Classification to the Training set

In [8]:
library(randomForest)
classifier = randomForest(x = training_set[-692],
                          y = training_set$Liked,
                          ntree = 10)

Warning message:
"package 'randomForest' was built under R version 3.4.3"randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.


Predicting the Test set results

In [9]:
y_pred = predict(classifier, newdata = test_set[-692])
y_pred

4   9  10  16  17  21  24  33  39  40  41  48  56  58  59  61  63  73  76  82 
  1   1   1   0   0   0   0   0   1   0   0   1   1   0   1   0   1   0   0   0 
 92  93  98  99 105 112 113 115 116 122 123 142 150 152 154 157 158 159 161 169 
  1   0   0   1   0   0   1   1   0   0   1   1   0   1   0   1   1   1   0   0 
182 183 184 188 190 191 193 199 202 203 210 211 217 222 228 239 240 250 251 255 
  1   0   0   0   1   1   1   1   1   0   0   0   1   0   1   0   1   0   1   1 
258 262 264 270 272 276 287 292 303 306 314 318 326 328 337 344 345 346 349 351 
  0   1   1   0   1   0   0   0   0   0   0   1   0   1   1   1   0   1   0   0 
353 361 363 364 370 375 395 396 397 399 412 413 415 416 430 433 445 446 453 456 
  0   0   0   1   0   1   1   0   0   1   1   0   0   0   0   1   1   0   1   0 
466 469 470 473 486 495 496 509 519 521 525 528 531 535 539 545 548 555 560 563 
  1   1   0   1   0   0   0   0   0   0   1   0   1   0   0   0   0   1   0   1 
568 570 574 583 586 591 598 606 613 614 618 625 628 633 634 639 641 647 648 653 
  1   0   0   0   1   1   0   1   0   0   0   1   0   0   0   0   1   0   1   1 
658 668 674 679 688 694 698 712 715 716 719 730 739 743 752 759 761 768 780 789 
  1   1   1   1   1   1   0   0   1   1   0   1   1   0   1   1   1   1   1   1 
795 807 809 811 817 818 821 844 848 849 853 855 863 868 874 882 890 891 892 894 
  0   0   1   1   0   1   0   0   1   0   1   1   1   0   0   0   1   1   1   0 
900 905 906 912 915 920 924 931 935 938 939 941 953 956 965 973 977 983 985 996 
  1   0   1   0   0   1   0   1   0   0   0   0   1   0   0   0   0   0   0   0 
Levels: 0 1

Making the Confusion Matrix

In [10]:
cm = table(test_set[, 692], y_pred)
cm

   y_pred
     0  1
  0 79 21
  1 30 70

In [12]:
accuracy = (79+70)/200
accuracy

[1] 0.745

We got an accuracy of almost 75%